In [ ]:
import requests
from requests import request
from requests.compat import urljoin, urlencode
from requests.sessions import Session
from bs4 import BeautifulSoup
import re

# Scrapping Mails

In [ ]:
s = Session()

url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https://www.naver.com/'
resp = s.request('get', url)
dom = BeautifulSoup(resp.text, 'html.parser')

In [ ]:
form = dom.select_one('form')
form.attrs['action'], form.attrs['method']

('https://nid.naver.com/nidlogin.login', 'POST')

In [ ]:
for el in form.select('[name]'):
    print(el.name, el.attrs['name'], el.attrs['value'] if el.has_attr('value') else None)

input localechange 
input dynamicKey 8g425Hjc3VSZOJVRnlC7aYOYcOn1JUP1nR5Noff3HEdhe0gnL0g2DX6SMooFDzMoegWq6HM-T0D9DqWFx47lMgXwNn25w-uihpjOcx9f8OM
input encpw 
input sessionKey 
input show_pk true
input wtoken 
input svctype 1
input bvsd 
input locale ko_KR
input url https://www.naver.com/
input enctp 1
input next_step false
input fbOff true
input smart_LEVEL 1
input encnm 
input id 
input pw None
input nvlong off


In [ ]:
# 브라우저 개발자 도구 > Application > Cookies > https://www.naver.com
c = '''

'''

In [ ]:
cookies = dict()

for line in c.splitlines():
    if len(line) > 0:
        cookies[line.split()[0]]=line.split()[1]

In [ ]:
url = 'https://mail.naver.com'

In [ ]:
# 쿠키를 이용해 로그인 한 후
re.search(r'<title>(.+)</title>', request('get', url, cookies=cookies).text).group(1)

'메일'

In [ ]:
# 받은메일함
url = 'https://mail.naver.com/v2/folders/0'
resp = s.request('get', url, cookies=cookies)
re.search(r'<title>(.+)</title>', resp.text).group(1)

'메일'

In [ ]:
# 메일 제목을 담을 list
mails = []

In [ ]:
url = 'https://mail.naver.com/json/list'

# 받은메일함
# 개발자 도구 > Network > Fetch/XHR 또는 JS로 오는 req/resp 확인
params = {
    'folderSN':'0',
    'page':'1',
    'viewMode':'time',
    'previewMode':'1',
    'sortField':'1',
    'sortType':'0',
    'u':'liebenholz98'
}

resp = request('POST', url, params=params, cookies=cookies)
resp.headers['content-type']

'text/plain;charset=UTF-8'

In [ ]:
for mail in resp.json()['mailData']:
    mails.append((mail['subject'], 'no'))
    print(mail['subject'])

In [ ]:
# 스팸메일함: 'folderSN':'5'
params = {
    'folderSN':'5',
    'page':'1',
    'viewMode':'time',
    'previewMode':'1',
    'sortField':'1',
    'sortType':'0',
    'u':'liebenholz98'
}

resp = request('POST', url, params=params, cookies=cookies)
resp.headers['content-type']

'text/plain;charset=UTF-8'

In [ ]:
for mail in resp.json()['mailData']:
    mails.append((mail['subject'], 'yes'))
    print(mail['subject'])

In [ ]:
mails # 받은메일함 + 스팸메일함

In [ ]:
from random import shuffle

shuffle(mails) # 메일함 shuffle
mails

In [ ]:
mailList = [] # 처리할 메일 리스트

# 마지막 4개의 메일 제목을 제외하고 삽입
for i in range(len(mails)-4):
    mailList.append((i+1, mails[i][0], mails[i][1]))

# 마지막 4개의 메일은 Tag 삭제 후 삽입
for i in range(4):
    mailList.append((len(mailList)+1, mails[i][0], None))

mailList

# Naive Bayse based Spam Filter

In [ ]:
C = list(set(x[-1] for x in mailList[:-1]))

In [ ]:
from nltk import download
download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.text import Text

def train(c, d):
    V = list()
    for x in d:
        for t in word_tokenize(x[1]):
            if t not in V:
                V.append(t)

    N = len(d)

    prior = {}
    condprob = {}
    for category in c:
        Nc = len(list(filter(lambda x:x[-1] == category, d)))
        prior[category] = Nc/N
        textc = '\n'.join([x[1] for x in d if x[-1] == category])
        textc = Text(word_tokenize(textc.lower()))
        for t in V:
            Tct = textc.count(t)
            if t not in condprob:
                condprob[t] = {}
            condprob[t][category] = (Tct+1)/(textc.vocab().N()+len(V))

    return V, prior, condprob

In [ ]:
Vocabulary, Prior, CondProb = train(C, mailList[:-4])
CondProb

In [ ]:
# '광고': {None: 0.003246753246753247,
#    'no': 0.0016313213703099511,
#    'yes': 0.03784570596797671} <- Yes의 비중이 압도적으로 높음

In [ ]:
from numpy import log

def test(c, V, prior, condprob, d):
    W = list()
    for t in word_tokenize(d.lower()):
        if t in V:
            W.append(t)

    score = dict()
    for category in C:
        score[category] = log(prior[category])
        for t in W:
            score[category] += log(condprob[t][category])

    return max(score, key=score.get), score

In [ ]:
# (광고) 성혼 정보회사 등급 테스트, 나의 등급은 몇 점일까?
test(C, Vocabulary, Prior, CondProb, mailList[-4][1])

<ipython-input-27-1f587da94c1f>:10: RuntimeWarning: divide by zero encountered in log
  score[category] = log(prior[category])


('yes',
 {None: np.float64(-inf),
  'no': np.float64(-79.90174524434953),
  'yes': np.float64(-66.50345053811999)})

In [ ]:
# [와디즈 노션대학교] &#39;1% 노션 템플릿&#39; 리뷰 이벤트 PPT 템플릿 보내드립니다!🎉
test(C, Vocabulary, Prior, CondProb, mailList[-3][1])

<ipython-input-27-1f587da94c1f>:10: RuntimeWarning: divide by zero encountered in log
  score[category] = log(prior[category])


('no',
 {None: np.float64(-inf),
  'no': np.float64(-115.01257586816887),
  'yes': np.float64(-127.87608959588287)})

In [ ]:
# 피카플레이 회원이 되신 것을 환영합니다!
test(C, Vocabulary, Prior, CondProb, mailList[-2][1])

<ipython-input-27-1f587da94c1f>:10: RuntimeWarning: divide by zero encountered in log
  score[category] = log(prior[category])


('no',
 {None: np.float64(-inf),
  'no': np.float64(-33.34800442339382),
  'yes': np.float64(-37.785238536362144)})

In [ ]:
# (광고) [클룩 단독] 에어서울 항공권 최대 98% 할인 중! 클룩 투어/렌터카 10% 할인까지 (~11/10)
test(C, Vocabulary, Prior, CondProb, mailList[-1][1])

<ipython-input-27-1f587da94c1f>:10: RuntimeWarning: divide by zero encountered in log
  score[category] = log(prior[category])


('yes',
 {None: np.float64(-inf),
  'no': np.float64(-131.19052709236036),
  'yes': np.float64(-116.19913274350338)})

In [ ]:
s.cookies.clear()